# National Institutes of Health

Data from the National Institutes of Health via their [RePORTER](https://projectreporter.nih.gov/reporter.cfm) system.

Data is obtained via their Batch RePORTER [ExPorter](https://reporter.nih.gov/exporter) for FY 1985-FY 2022 (federal fiscal year runs from October 1 to September 30). Data after that is obtained via their [API](https://api.reporter.nih.gov/).

[Data Dictionary](https://api.reporter.nih.gov/documents/Data%20Elements%20for%20RePORTER%20Project%20API_V2.pdf)

## Setup and Prep

In [1]:
import pandas as pd
import datetime
from typing import List, Dict, Any
import requests
import time
from tqdm.notebook import tqdm
import json
import io
import zipfile
import polars as pl

## USE ONLY FOR DEV
import requests_cache
requests_cache.install_cache("dev_http_cache.sqlite", backend="sqlite", expire_after=datetime.timedelta(days=1))
## USE ONLY FOR DEV

pd.set_option("display.max_columns", None)

In [2]:
fy_years = "2010 - now"
"""
fy_years is a parameter that is used to filter the data by fiscal year. 
It can accept a single year, a range of years, or a list of years. 
It can also accept the values 'ALL' for all valid dates, or the value 'now' as a single year
or as the end date of a range.
The format for the parameter is as follows:
- A single year: "2019"
- A range of years: "2018-2023"
- The value 'ALL' for all valid dates
- "2018 - now"
- A comma-separated list of years: "2018, 2019, 2020"
"""
use_batch = True

In [3]:
# Validation and formatting for parameters


def validate_and_transform_fy_years(fy_years: str) -> List[int]:
    """
    Validates and transforms fiscal year input into a list of integers representing the fiscal years.

    Args:
        fy_years (str): The fiscal year input. It can be one of the following:
            - 'ALL': Returns a list of all valid fiscal years from 1985 to the current fiscal year.
            - 'now': Returns a list containing only the current fiscal year.
            - 'start_year-end_year': Returns a list of fiscal years from start_year to end_year (inclusive).
              The end_year can be 'now' to represent the current fiscal year.
            - 'year1,year2,...': Returns a list of specific fiscal years provided as comma-separated values.

    Returns:
        List[int]: A list of integers representing the fiscal years.

    Raises:
        ValueError: If the input is invalid or outside the valid range of fiscal years.

    """
    current_year = datetime.datetime.now().year
    min_year = 1985
    max_year = (
        current_year + 1 if datetime.datetime.now().month >= 10 else current_year
    )  # Next Fiscal year starts in October

    if fy_years == "ALL":
        return list(range(min_year, max_year + 1))
    elif fy_years == "now":
        return [max_year]
    elif "-" in fy_years:
        start_year, end_year = fy_years.split("-")
        start_year = int(start_year.strip())
        if str(end_year).strip().lower() == "now":
            end_year = max_year
        else:
            end_year = int(end_year.strip())
        if start_year < min_year or end_year > max_year:
            raise ValueError(
                f"Invalid year range. Valid range is {min_year}-{max_year}."
            )
        return list(range(start_year, end_year + 1))
    elif "," in fy_years:
        years = fy_years.split(",")
        years = [int(year.strip()) for year in years]
        if any(year < min_year or year > max_year for year in years):
            raise ValueError(f"Invalid year. Valid range is {min_year}-{max_year}.")
        return years
    else:
        year = int(fy_years.strip())
        if year < min_year or year > max_year:
            raise ValueError(f"Invalid year. Valid range is {min_year}-{max_year}.")
        return [year]


FY_YEARS = validate_and_transform_fy_years(fy_years)

In [4]:
## Preparing derivative parameters: batch_years and api_years

min_batch_year = 1985
max_batch_year = 2022
batch_elligible_years = list(range(min_batch_year, max_batch_year + 1))

if use_batch:
    batch_years = [year for year in FY_YEARS if year in batch_elligible_years]
    api_years = [year for year in FY_YEARS if year not in batch_elligible_years]
else:
    batch_years = []
    api_years = FY_YEARS

## Retrieval

### Batch Retrieval

If years are <= 2022, and batch mode's been turned on, it'll go and get the Batch RePORTER data. If disabled or years are more recent, it'll go and get the data via the API.

In [5]:
def get_nih_batch_data(years: List[int]) -> pd.DataFrame:
    """
    Get NIH RePORTER data for the specified fiscal years using the batch download feature.

    Args:
        years (List[int]): A list of fiscal years for which to get the data.

    Returns:
        pd.DataFrame: A DataFrame containing the NIH RePORTER data for the specified fiscal years.

    """
    nih_batch_projects_csvzip_url = "https://reporter.nih.gov/exporter/projects/download/{}"

    batch_dfs = []
    progress_bar = tqdm(total=len(years), desc="Downloading NIH batch files")
    for year in years:

        try:
            url = nih_batch_projects_csvzip_url.format(year)
            response = requests.get(url)
            _crawled_at = datetime.datetime.utcnow()
            zip_file = zipfile.ZipFile(io.BytesIO(response.content))
            files = zip_file.namelist()

            # unzip and load 
            with zip_file.open(files[0]) as f:
                df = pd.read_csv(f,
                    low_memory=False,
                    encoding="latin1",
                    on_bad_lines="warn",
                )

                ## Add our own metadata fields to the dataframe
                df['_batch_fy'] = year
                df["_crawled_at"] = _crawled_at
                df["_source_url"] = url
                
            batch_dfs.append(df)
            progress_bar.update(1)
            time.sleep(1)
        except Exception as e:
            raise ValueError(f"Error reading batch file for year {year}: {e}")
    progress_bar.close()
    return pd.concat(batch_dfs, ignore_index=True)

def clean_batch_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean the NIH batch input DataFrame by dropping duplicate rows based on the '
    APPLICATION_ID' column, and formatting specified date columns to datetime objects.
    
    Args:
        df (pd.DataFrame): The input DataFrame to be cleaned.
        
    Returns:
        pd.DataFrame: The cleaned DataFrame.
    """
    df.drop_duplicates(subset="APPLICATION_ID", inplace=True)

    # Format date columns
    date_columns = [
        "AWARD_NOTICE_DATE",
        "BUDGET_START",
        "BUDGET_END",
        "PROJECT_START",
        "PROJECT_END",
    ]
    for column in date_columns:
        df[column] = pd.to_datetime(
            df[column], format="mixed", errors="coerce"
        )
    
    return df
    
    

if len(batch_years) > 0:
    batch_df = get_nih_batch_data(batch_years)
    batch_df = clean_batch_df(batch_df)

/tmp/ipykernel_17918/2823648914.py:27: ParserWarning: Skipping line 46322: expected 46 fields, saw 47

  df = pd.read_csv(f,
/tmp/ipykernel_17918/2823648914.py:27: ParserWarning: Skipping line 55941: expected 46 fields, saw 47

  df = pd.read_csv(f,


#### Attempting to improve performance with Polars

Lots of time is spent waiting for loading this in Pandas. Does Polars make it any faster?

### API Retrieval

Accessing data from RePORTER API

In [56]:
def call_nih_api(
    years: List[int] = None,
    offset: int = 0,
    result_limit: int = 50,
    search_id: str = None,
    payload: Dict[Any, Any] = None,
) -> Dict[Any, Any]:
    """
    Calls the NIH RePORTER API to get project data for the specified fiscal years.

    Args:
        years (List[int]): A list of fiscal years for which to get project data.
            Used on initial search; subsequent searches can use search_id
            to continue retrieving results from the same search session.
        offset (int, optional): The offset for the API request. Defaults to 0.
        result_limit (int, optional): The maximum number of results to return. Min = 1, Max = 500
        search_id (str, optional): The search ID to continue retrieving results from a previous search session.
            If provided, the years will be ignored
        payload (Dict[Any, Any], optional): Allows for manual specification of the complete search payload.
            Overrides defaults provided while using any other parameters.
            Documentation at https://api.reporter.nih.gov/?urls.primaryName=V2.0

    Returns:
        Dict[Any, Any]: The response from the API request.

    Raises:
        ValueError: If the API request fails or returns an error.

    """
    api_url = "https://api.reporter.nih.gov/v2/projects/search"
    if payload is None:
        payload = {
            "offset": offset,
            "limit": result_limit,
            "sort_field": "project_start_date",
            "sort_order": "desc",
        }
        if search_id is not None:
            payload["criteria"] = {"search_id": search_id}
        elif years is not None:
            payload["criteria"]={"fiscal_years": years}
        else:
            raise ValueError("No search criteria provided. Needs xor(fiscal_years or a search id when not providing a fully-specified api payload.")
    

    headers = {"Content-Type": "application/json"}
    response = requests.post(api_url, data=json.dumps(payload), headers=headers)
    if response.status_code != 200:
        raise ValueError(
            f"API request failed with status code {response.status_code}. Error message: {response.text}"
        )
    response_json = response.json()
    if "error" in response_json:
        raise ValueError(f"API request returned an error: {response_json['error']}.")
    return response_json


c = call_nih_api(years=[2024], offset=0, result_limit=1)

In [59]:
c['meta']

{'search_id': 'YqUVCzrEXkCxAZxIHl_V8A',
 'total': 11625,
 'offset': 0,
 'limit': 1,
 'sort_field': 'project_start_date',
 'sort_order': 'desc',
 'sorted_by_relevance': False,
 'properties': {'URL': 'https:/reporter.nih.gov/search/YqUVCzrEXkCxAZxIHl_V8A/projects'}}

In [54]:
def get_nih_api_data(years: List[int]) -> pd.DataFrame:
    """
    Retrieves data from the NIH API for projects funded or active during a list of fiscal years.

    Args:
        years (List[int]): A list of fiscal years for which data is to be retrieved.

    Returns:
        pd.DataFrame: A DataFrame containing the API results for the specified fiscal years.
    """

    # TODO implement an approach to overcome the 14,999 offset limit.
    offset = 0
    api_results = []
    response = call_nih_api(years, offset=0)
    _crawled_at = datetime.datetime.utcnow()
    total_results = response["meta"]["total"]
    api_results.extend(response["results"])
    current_count = len(api_results)
    progress_bar = tqdm(total=total_results, desc="Downloading NIH API data")
    progress_bar.update(current_count)

    while current_count < total_results:
        offset += response["meta"]["limit"]
        while (datetime.datetime.utcnow() - _crawled_at).total_seconds() < 1:
            time.sleep(0.1)
        response = call_nih_api(api_years, offset=offset)
        _crawled_at = datetime.datetime.utcnow()
        if response["meta"]["total"] != total_results:
            raise ValueError("Total results count changed during API request.")
        api_results.extend(response["results"])
        current_count = len(api_results)
        progress_bar.update(len(response["results"]) )

    progress_bar.close()

    api_df = pd.json_normalize(api_results)
    return api_df

api_df = get_nih_api_data([2024])


NameError: name 'call_nih_api' is not defined

#### Making sense of RePORTER/ExPORTER Data:

> Active Projects: Unless specified, RePORTER searches for “Active Projects;” the most recent record of projects that have not yet reached their end date (in the case of extramural grants and contracts) or intramural projects from the most recent database of annual reports from the NIH intramural programs. (Information on intramural projects is updated in RePORTER at the close of each fiscal year.) This means that the results of “Active Projects” searches may include projects that were funded with an earlier fiscal year’s appropriation but are still active (e.g., extramural awards in a no-cost extension period) or not yet updated by the end-of-year intramural project report.

TODO: API has a maximum offset of 14,999 (regardless of page size). We will need an altrenate way to page through the data, either through a pagination mechanism or by limiting some aspect (e.g. iterating over possible funding vehicles).

##### Mapping API fields to ExPORTER fields

The API fields are documented [here](https://api.reporter.nih.gov/swagger/index.html). The ExPORTER fields are documented [here](https://exporter.nih.gov/ExPORTER_Catalog.aspx). NIH provides an [api call and crosswalk guide](https://api.reporter.nih.gov/documents/Sample%20RePORTER%20API%20Request%20Matching%20ExPORTER%20Project%20Files.pdf) to go from API fields to ExPORTER fields.

TODO: build a weekly updater based on the reporter api -> exporter data structure.

#### Concerns to address
- Abstracts: how much more data comes along with abstracts that would be fundamentally useful for us here?
- API and "Active" projects only: what's the configuration to allow non-active projects
- Offset limit of 14,999: how can we page through the data

##### Handling API offset limits:

The API has a maximum offset of 14,999 (regardless of page size). We will need an altrenate way to page through the data, regardless of age.

In practice, this is close to a practical limit of 29,000 results as we can reverse the sort order and page through the data in reverse.

This still does not let us reach the >50,000 results in RP (Non-SBIR/STTR Research Projects).

Have tried:

  - Using result limits of 500; fewer pages (faster), still hits the offset limit
  - Requesting 1 year-at-a-time; ~80-90k grants year, still exceeds target
  - Requesting projects by their funding vehicle (and by project); ~50k projects / yr
  - Date-based filtering for fiscal years

Additional strategies to try:
- 



##### The RePORTER -> ExPORTER Query

From [api call to export crosswalk guide](https://api.reporter.nih.gov/documents/Sample%20RePORTER%20API%20Request%20Matching%20ExPORTER%20Project%20Files.pdf).

```json
{
"criteria":
{
"funding_mechanism":["SB","RP","RC","OR","TR","TI","CO","IM","IAA","SRDC","Other"],
"newly_added_projects_only": true,
},
"include_fields": [
"ApplId","SubprojectId","FiscalYear","ProjectNum","ProjectSerialNum","Organization", "OrganizationType",
"AwardType", "ActivityCode", "AwardAmount", "ProjectNumSplit", "PrincipalInvestigators", "ProgramOfficers",
"AgencyIcAdmin", "AgencyIcFundings","CongDist", "ProjectStartDate","ProjectEndDate","FullFoa","FullStudySection",
"AwardNoticeDate", "CoreProjectNum","PrefTerms", "ProjectTitle", "PhrText","SpendingCategoriesDesc", "ArraFunded",
"BudgetStart", "BudgetEnd","CfdaCode","FundingMechanism","DirectCostAmt","IndirectCostAmt"
],
"offset":0,
"limit":25,
"sort_field":"project_start_date",
"sort_order":"desc"
}

```

##### Approach 1: subset by years and funding mechanisms

We almost certainly need to find a way to subdivide approaches we use to query the API. Individual years have > 15,000 records.

How many results are there for each year we're looking at? *not matching criteria for export crosswalk; just looking at result total*

In [69]:
total_yearly_results = []
for year in FY_YEARS:
    year_result = call_nih_api(years=[year], offset=0)
    time.sleep(1)
    year_total = year_result['meta']['total']
    total_yearly_results.append({year: int(year_total)})

In [70]:
total_yearly_results__table = pd.DataFrame(
    [(list(d.keys())[0], list(d.values())[0]) for d in total_yearly_results],
    columns=["Year", "Total Projects"],
)
total_yearly_results__table

,Year,Total Projects
0,2010,94009
1,2011,83645
2,2012,78989
3,2013,77056
4,2014,76188
5,2015,73436
6,2016,72491
7,2017,73144
8,2018,80826
9,2019,79463


Are our counts different if we use our funding mechanism criteria?

In [71]:
exporter_payload = {
"criteria":
{
"funding_mechanism":["SB","RP","RC","OR","TR","TI","CO","IM","IAA","SRDC","Other"],
# "newly_added_projects_only": true,
},
"include_fields": [
"ApplId","SubprojectId","FiscalYear","ProjectNum","ProjectSerialNum","Organization", "OrganizationType",
"AwardType", "ActivityCode", "AwardAmount", "ProjectNumSplit", "PrincipalInvestigators", "ProgramOfficers",
"AgencyIcAdmin", "AgencyIcFundings","CongDist", "ProjectStartDate","ProjectEndDate","FullFoa","FullStudySection",
"AwardNoticeDate", "CoreProjectNum","PrefTerms", "ProjectTitle", "PhrText","SpendingCategoriesDesc", "ArraFunded",
"BudgetStart", "BudgetEnd","CfdaCode","FundingMechanism","DirectCostAmt","IndirectCostAmt"
],
"offset":0,
"limit":1,
"sort_field":"project_start_date",
"sort_order":"asc"
}

In [72]:
filter_count_comparison = pd.DataFrame(columns=['year', 'unfiltered_count', 'funding_mechanism_filter_true_count'])

pb = tqdm(total=len(FY_YEARS)*2, desc="downloading o(n)*2 data from NIH API")
for year in FY_YEARS:
    year_result = call_nih_api(years=[year], offset=0, result_limit=1)
    time.sleep(1)
    pb.update(1)
    year_total = year_result['meta']['total']
    payload = exporter_payload
    payload['criteria']['fiscal_years'] = [year]
    year_result_filtered = call_nih_api(payload=payload)
    time.sleep(1)
    pb.update(1)
    year_total_filtered = year_result_filtered['meta']['total']
    filter_count_comparison = pd.concat([filter_count_comparison, pd.DataFrame({'year': [year], 'unfiltered_count': [int(year_total)], 'funding_mechanism_filter_true_count': [int(year_total_filtered)]})], ignore_index=True)

pb.close()
filter_count_comparison

downloading o(n)*2 data from NIH API:   0%|          | 0/30 [00:00<?, ?it/s]

,year,unfiltered_count,funding_mechanism_filter_true_count
0,2010,94009,92894
1,2011,83645,82444
2,2012,78989,77765
3,2013,77056,75719
4,2014,76188,74928
5,2015,73436,72163
6,2016,72491,71004
7,2017,73144,71806
8,2018,80826,79535
9,2019,79463,78224


Truly not that drastic of a difference and they all exceed 15,000. We need to further subdivide.

Will breaking down by year AND funding mechanism do the trick? Ideally this should give us meaningful margin under 15,000 so that there's room for the crawler to adapt to funding shifts from year-to-year, but lets just start with what we have.

In [26]:
funding_mechnisms = ["SB","RP","RC","OR","TR","TI","CO","IM","IAA","SRDC","Other"]
pb = tqdm(total=len(funding_mechnisms)*len(FY_YEARS), desc="downloading o(n)*11 data from NIH API")

# taking another approach here: stuffing the results into a dictionary and then converting to a dataframe

fm_responses  = {}

for year in FY_YEARS:
    year_responses = {}
    for mechanism in funding_mechnisms:
        payload = exporter_payload
        payload['criteria']['fiscal_years'] = [year]
        payload['criteria']['funding_mechanism'] = [mechanism]
        year_result_filtered = call_nih_api(payload=payload)
        time.sleep(1)
        pb.update(1)
        year_responses[mechanism] = year_result_filtered
    fm_responses[year] = year_responses

pb.close()




downloading o(n)*11 data from NIH API:   0%|          | 0/165 [00:00<?, ?it/s]

In [31]:
annual_fm_breakdown = pd.DataFrame([[year, sibling, fm_responses[year][sibling]['meta']['total']] for year in fm_responses for sibling in fm_responses[year]], columns = ['year', 'funding_mechanism', 'count']).pivot(index='year', columns='funding_mechanism', values='count')
annual_fm_breakdown

funding_mechanism,CO,IAA,IM,OR,Other,RC,RP,SB,SRDC,TI,TR
year,,,,,,,,,,,
2010,80,225,3520,8789,6626,15711,50387,1917,108,2304,3227
2011,3,249,3308,7934,6653,13988,43330,1620,96,2234,3029
2012,0,175,3448,7920,6591,9169,43440,1724,97,2151,3050
2013,4,148,3123,7836,6491,9043,42266,1560,105,2046,3097
2014,2,145,3148,7795,6279,8681,41938,1768,96,2008,3068
2015,0,154,3298,7950,2914,8982,41893,1706,88,1993,3185
2016,0,148,3182,7701,1372,8969,42370,1835,81,2020,3326
2017,1,149,3149,7573,515,9250,42889,1943,80,2495,3762
2018,2,135,3072,8056,653,13613,46179,2243,10,2043,3529


In [37]:
merged_df = pd.merge(filter_count_comparison, annual_fm_breakdown, left_on='year', right_index=True,)
merged_df['fm_counts_equivelent']=merged_df[funding_mechnisms].sum(axis=1)==merged_df['funding_mechanism_filter_true_count']
merged_df

,year,unfiltered_count,funding_mechanism_filter_true_count,CO,IAA,IM,OR,Other,RC,RP,SB,SRDC,TI,TR,fm_counts_equivelent
0,2010,94009,92894,80,225,3520,8789,6626,15711,50387,1917,108,2304,3227,True
1,2011,83645,82444,3,249,3308,7934,6653,13988,43330,1620,96,2234,3029,True
2,2012,78989,77765,0,175,3448,7920,6591,9169,43440,1724,97,2151,3050,True
3,2013,77056,75719,4,148,3123,7836,6491,9043,42266,1560,105,2046,3097,True
4,2014,76188,74928,2,145,3148,7795,6279,8681,41938,1768,96,2008,3068,True
5,2015,73436,72163,0,154,3298,7950,2914,8982,41893,1706,88,1993,3185,True
6,2016,72491,71004,0,148,3182,7701,1372,8969,42370,1835,81,2020,3326,True
7,2017,73144,71806,1,149,3149,7573,515,9250,42889,1943,80,2495,3762,True
8,2018,80826,79535,2,135,3072,8056,653,13613,46179,2243,10,2043,3529,True
9,2019,79463,78224,14,123,3073,8547,681,10283,47568,2222,94,1998,3621,True


**The set of results is too great to rely on funding mechanism alone**

However, we can now confirm that funding mechanism is a required enum (no nulls when matching exporter, and only one funding mechanism per project). We can use this to further subdivide our queries, but we'll need another strategy to get under 15,000.

**HOWEVER**, only 2 categories (RC = Research Centers ) and (RP = Non-SBIR/STTR Research Projects) ever exceed 15k records.

In [80]:
batch_df.groupby('FY')['APPLICATION_ID'].count().reset_index(drop=True)


0     93840
1     83642
2     78988
3     77035
4     76182
5     73427
6     72489
7     73133
8     80796
9     79462
10    82411
11    82908
12    83730
Name: APPLICATION_ID, dtype: int64

In [81]:
filter_count_comparison['batch_counts'] = batch_df.groupby('FY')['APPLICATION_ID'].count().reset_index(drop=True)

In [82]:
filter_count_comparison

,year,unfiltered_count,funding_mechanism_filter_true_count,batch_counts
0,2010,94009,92894,93840.0
1,2011,83645,82444,83642.0
2,2012,78989,77765,78988.0
3,2013,77056,75719,77035.0
4,2014,76188,74928,76182.0
5,2015,73436,72163,73427.0
6,2016,72491,71004,72489.0
7,2017,73144,71806,73133.0
8,2018,80826,79535,80796.0
9,2019,79463,78224,79462.0


Based on the above, it seems like FY in the api is damn close to batch exports; which is to say without the pre-existing filters.

We may have no choice but to page through the data chronologically based on the start date of the project and discard the batches.

Luckily it appears our earliest `award_notice_date` for a FY is a few months before the start of the FY, so we can use that as a filter.

| FY | max(award_notice_date) | min(award_notice_date) | 
|----|------------------------|------------------------|
| 2010 | 2014-11-19 | 2009-05-01 | 
| 2011 | 2015-01-22 | 2010-07-12 | 
| 2012 | 2016-08-31 | 2011-10-11 | 
| 2013 | 2016-09-28 | 2012-07-05 | 
| 2014 | 2017-09-18 | 2013-09-21 | 
| 2015 | 2018-09-18 | 2014-10-15 | 
| 2016 | 2020-08-11 | 2015-04-20 | 
| 2017 | 2020-11-10 | 2016-08-02 | 
| 2018 | 2020-09-11 | 2017-08-29 | 
| 2019 | 2023-08-21 | 2016-11-22 | 
| 2020 | 2023-09-01 | 2019-03-08 | 
| 2021 | 2023-09-09 | 2020-05-14 | 
| 2022 | 2023-09-08 | 2020-12-09 | 

The closer we get to now, the more likely it is to have some degree of lag. That said, it should still be sufficient to allow us to page through the data.

In [101]:
min_max_values = batch_df.groupby('FY')[['AWARD_NOTICE_DATE', 'BUDGET_START', 'BUDGET_END', 'PROJECT_START', 'PROJECT_END']].agg({'min', 'max'})


In [102]:
batch_df['AWARD_NOTICE_DATE'].value_counts()

AWARD_NOTICE_DATE
2018-09-19    1653
2018-09-11    1250
2018-09-14    1213
2017-06-30    1119
2018-09-13    1105
              ... 
2023-08-31       1
2022-10-19       1
2022-11-08       1
2022-11-25       1
2023-02-11       1
Name: count, Length: 4677, dtype: int64

In [108]:
import pandas as pd
from datetime import timedelta
notice_date = batch_df['AWARD_NOTICE_DATE']
earliest_notice_date = notice_date.min()
latest_notice_date = notice_date.max()

# Create a new DataFrame to store the results
awards_count = pd.DataFrame(columns=['Date', 'Count'])

# Iterate over each day in the range
for date in pd.date_range(earliest_notice_date, latest_notice_date + timedelta(days=1)):
    # Calculate the end date of the 14-day window
    end_date = date + timedelta(days=14)
    
    # Filter the batch_df DataFrame based on the date range
    filtered_df = batch_df[(batch_df['AWARD_NOTICE_DATE'] >= date) & (batch_df['AWARD_NOTICE_DATE'] < end_date)]
    
    # Count the number of awards in the filtered DataFrame
    count = len(filtered_df)
    
    # Concatenate the filtered DataFrame with the awards_count DataFrame
    awards_count = pd.concat([awards_count, pd.DataFrame({'Date': [date], 'Count': [count]})], ignore_index=True)

# Print the awards_count DataFrame
print(awards_count)


/tmp/ipykernel_17918/4079622769.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  awards_count = pd.concat([awards_count, pd.DataFrame({'Date': [date], 'Count': [count]})], ignore_index=True)


           Date Count
0    2009-05-01     1
1    2009-05-02     0
2    2009-05-03     0
3    2009-05-04     0
4    2009-05-05     0
...         ...   ...
5241 2023-09-06    12
5242 2023-09-07    12
5243 2023-09-08     7
5244 2023-09-09     1
5245 2023-09-10     0

[5246 rows x 2 columns]


In [111]:
max(awards_count['Count'])

10075

Over a 14-day period, the maximum number of grant notices has been 10,075. This is a viable sign that we can set our maximum date range to 14 days.